In [24]:
# Load textures from freesound

FREESOUND_API_KEY = 'JI0jcelpAEeqZOL0OBE4dJrTD8yFiMOjo5zEQR9u'  # Please replace by your own Freesound API key
FILES_DIR = 'oggs'  # Place where to store the downloaded diles. Will be relative to the current folder.
DATAFRAME_FILENAME = 'testdata/dac-train.xlsx'  # File where we'll store the metadata of our sounds collection
FREESOUND_STORE_METADATA_FIELDS = ['id', 'name', 'username', 'previews', 'license', 'tags']  # Freesound metadata properties to store
FREESOUND_QUERIES = [
    {
        'query': 'helicopter',
        'filter': "duration:[6 TO 30]",
        'num_results': 50,
    },
{
        'query': 'radio',
        'filter': "duration:[6 TO 30]",
        'num_results': 50,
    },
{
        'query': 'rain',
        'filter': "duration:[6 TO 30]",
        'num_results': 50,
    },
{
        'query': 'supersaw',
        'filter': "duration:[6 TO 30]",
        'num_results': 50,
    },
]
import os
import pandas as pd
import numpy as np
import freesound
from tqdm import tqdm
import shutil


def query_freesound(query, filter, freesound_client, num_results=10):
    """Queries freesound with the given query and filter values.
    If no filter is given, a default filter is added to only get sounds shorter than 30 seconds.
    """
    if filter is None:
        filter = 'duration:[0 TO 30]'  # Set default filter
    pager = freesound_client.text_search(
        query = query,
        filter = filter,
        fields = ','.join(FREESOUND_STORE_METADATA_FIELDS),
        group_by_pack = 1,
        page_size = num_results
    )
    return [sound for sound in pager]
    
def retrieve_sound_preview(sound, freesound_client, directory):
    """Download the high-quality OGG sound preview of a given Freesound sound object to the given directory.
    """
    return freesound.FSRequest.retrieve(
        sound.previews.preview_hq_ogg,
        freesound_client,
        os.path.join(directory, sound.name.replace(' ', '_').replace("/", ""))
    )

def force_mkdir(directory):
    if os.path.exists(directory):
        shutil.rmtree(directory)
    os.makedirs(directory)


# Download from freesound

In [35]:
freesound_client = freesound.FreesoundClient()
freesound_client.set_token(FREESOUND_API_KEY)

force_mkdir(FILES_DIR)

# Do all queries and concatenate the results in a single list of sounds
sounds = sum([query_freesound(query['query'], query['filter'], freesound_client, query['num_results']) for query in FREESOUND_QUERIES],[])

sound_dict = {query["query"]: query_freesound(query['query'], query['filter'], freesound_client, query['num_results']) for query in FREESOUND_QUERIES}

mapping = dict()
# Reverse sound_dict
for key, value in sound_dict.items():
    for s in value:
        mapping[s.name.replace(" ", "_").replace("/", "")] = key


# Download the sounds and save them to FILES_DIR folder
for count, sound in enumerate(tqdm(sounds, desc='Downloading sounds')):
    retrieve_sound_preview(sound, freesound_client, FILES_DIR)

# Convert to WAV (5s long)

In [66]:

import essentia.standard as es
import os


force_mkdir("wavs")
mapping2 = dict()
total_length = 0
for filename in os.listdir("oggs"):
    ogg_path = os.path.join("oggs", filename)
    # Change extension to WAV
    wav_path = os.path.join("wavs", filename)

    # Load OGG file using Essentia
    audio = es.MonoLoader(filename=ogg_path)()
    
    # Trim to first 5 seconds (Essentia loads audio as a NumPy array with a sample rate)
    sample_rate = 44100  # Default sample rate in Essentia

    total_length += len(audio) / sample_rate
    # Trim audio into 5s frames:
    for i in range(0, len(audio), 5 * sample_rate):
        trimmed_audio = audio[i:i + 5 * sample_rate]
        if len(trimmed_audio) < 5 * sample_rate:
            continue
        # Save as WAV. include i right before the extension
        path = wav_path + f"_{i}.wav"
        
        es.MonoWriter(filename=path, format='wav')(trimmed_audio)
        mapping2[path] = mapping[filename]
        print(f"Processed: {filename} -> {wav_path}")



Processed: Lo-fi_AMFM_radio_(Two_stations_mixed).wav -> wavs/Lo-fi_AMFM_radio_(Two_stations_mixed).wav
Processed: urban_night_rain_1 -> wavs/urban_night_rain_1
Processed: urban_night_rain_1 -> wavs/urban_night_rain_1
Processed: urban_night_rain_1 -> wavs/urban_night_rain_1
Processed: urban_night_rain_1 -> wavs/urban_night_rain_1
Processed: airplane,_commercial_jet,_taxiing,_close_persspective_with_helicopter_flying_in_background -> wavs/airplane,_commercial_jet,_taxiing,_close_persspective_with_helicopter_flying_in_background
Processed: airplane,_commercial_jet,_taxiing,_close_persspective_with_helicopter_flying_in_background -> wavs/airplane,_commercial_jet,_taxiing,_close_persspective_with_helicopter_flying_in_background
Processed: airplane,_commercial_jet,_taxiing,_close_persspective_with_helicopter_flying_in_background -> wavs/airplane,_commercial_jet,_taxiing,_close_persspective_with_helicopter_flying_in_background
Processed: Roland_Super_Saw_Staccato_C.wav -> wavs/Roland_Super_Sa

[wav @ 0x449352e0] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x449352e0] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x449352e0] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x449352e0] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x449352e0] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x449352e0] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x449352e0] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x449352e0] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. 

Processed: Dripping_rain.wav -> wavs/Dripping_rain.wav
Processed: Dripping_rain.wav -> wavs/Dripping_rain.wav
Processed: Dripping_rain.wav -> wavs/Dripping_rain.wav
Processed: baby_radio_1.mp3 -> wavs/baby_radio_1.mp3
Processed: radio11.wav -> wavs/radio11.wav
Processed: radio11.wav -> wavs/radio11.wav
Processed: Rain_closed_window -> wavs/Rain_closed_window
Processed: Rain_closed_window -> wavs/Rain_closed_window
Processed: Rain_closed_window -> wavs/Rain_closed_window
Processed: Rain_closed_window -> wavs/Rain_closed_window
Processed: Radio_Noise_&_Mod_Rez_2.aif -> wavs/Radio_Noise_&_Mod_Rez_2.aif
Processed: Radio_Noise_&_Mod_Rez_2.aif -> wavs/Radio_Noise_&_Mod_Rez_2.aif
Processed: old-radio-noise-switchon-ukw.wav -> wavs/old-radio-noise-switchon-ukw.wav
Processed: old-radio-noise-switchon-ukw.wav -> wavs/old-radio-noise-switchon-ukw.wav
Processed: old-radio-noise-switchon-ukw.wav -> wavs/old-radio-noise-switchon-ukw.wav
Processed: room-tone_wind_rain_10_200216_0114.wav -> wavs/room-

[wav @ 0x448b0fa0] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x448b0fa0] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x448b0fa0] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x43ac1fa0] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x43ac1fa0] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x43ac1fa0] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x448b0fa0] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x448b0fa0] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. 

Processed: Taste_Bass_Supersaw_Synth_F#_4_Measures -> wavs/Taste_Bass_Supersaw_Synth_F#_4_Measures
Processed: Taste_Bass_Supersaw_Synth_F#_4_Measures -> wavs/Taste_Bass_Supersaw_Synth_F#_4_Measures
Processed: Taste_Bass_Supersaw_Synth_F#_4_Measures -> wavs/Taste_Bass_Supersaw_Synth_F#_4_Measures
Processed: airplane,_commercial_jet,_taxiing_with__helicopter_flying_overhead -> wavs/airplane,_commercial_jet,_taxiing_with__helicopter_flying_overhead
Processed: airplane,_commercial_jet,_taxiing_with__helicopter_flying_overhead -> wavs/airplane,_commercial_jet,_taxiing_with__helicopter_flying_overhead
Processed: airplane,_commercial_jet,_taxiing_with__helicopter_flying_overhead -> wavs/airplane,_commercial_jet,_taxiing_with__helicopter_flying_overhead
Processed: Helicopter_Overhead_SFX.wav -> wavs/Helicopter_Overhead_SFX.wav
Processed: Rain_Drips_in_a_bucket.mp3 -> wavs/Rain_Drips_in_a_bucket.mp3
Processed: Rain_Drips_in_a_bucket.mp3 -> wavs/Rain_Drips_in_a_bucket.mp3
Processed: Rain_Drips_i

...

[wav @ 0x4356f360] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x4356f360] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x4356f360] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x43ab1600] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x43ab1600] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x43ab1600] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x448a2fc0] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x448a2fc0] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. 

Processed: OH-58_Helicopter_Loop.flac -> wavs/OH-58_Helicopter_Loop.flac
Processed: rain.wav -> wavs/rain.wav
Processed: Helicopter_over_the_beach_today.mp3 -> wavs/Helicopter_over_the_beach_today.mp3
Processed: Helicopter_over_the_beach_today.mp3 -> wavs/Helicopter_over_the_beach_today.mp3
Processed: Helicopter_over_the_beach_today.mp3 -> wavs/Helicopter_over_the_beach_today.mp3
Processed: 1977_events_01.mp3 -> wavs/1977_events_01.mp3
Processed: Rain-Light-Loopable.wav -> wavs/Rain-Light-Loopable.wav
Processed: Short_Rain.wav -> wavs/Short_Rain.wav
Processed: Short_Rain.wav -> wavs/Short_Rain.wav
Processed: Various_Helicopter_Loops -> wavs/Various_Helicopter_Loops
Processed: Various_Helicopter_Loops -> wavs/Various_Helicopter_Loops
Processed: Various_Helicopter_Loops -> wavs/Various_Helicopter_Loops


[wav @ 0x4488fba0] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x4357aea0] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x4357aea0] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x4357aea0] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x4357aea0] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x4488fba0] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x4357aea0] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x447b8d60] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. 

Processed: rain_-_gutter_-_outside_laudry_room_window.wav -> wavs/rain_-_gutter_-_outside_laudry_room_window.wav
Processed: rain_-_gutter_-_outside_laudry_room_window.wav -> wavs/rain_-_gutter_-_outside_laudry_room_window.wav
Processed: rain_-_gutter_-_outside_laudry_room_window.wav -> wavs/rain_-_gutter_-_outside_laudry_room_window.wav
Processed: rain_-_gutter_-_outside_laudry_room_window.wav -> wavs/rain_-_gutter_-_outside_laudry_room_window.wav
Processed: rain_-_gutter_-_outside_laudry_room_window.wav -> wavs/rain_-_gutter_-_outside_laudry_room_window.wav
Processed: rain_against_window -> wavs/rain_against_window
Processed: rain_against_window -> wavs/rain_against_window
Processed: rain_against_window -> wavs/rain_against_window
Processed: rain_against_window -> wavs/rain_against_window
Processed: RAIN_STICK_A_001.wav -> wavs/RAIN_STICK_A_001.wav
Processed: radio_in_room.wav -> wavs/radio_in_room.wav
Processed: radio_in_room.wav -> wavs/radio_in_room.wav
Processed: radio_in_room.wav

[wav @ 0x434d5740] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x434d5740] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x434d5740] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x434d5740] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x434d5740] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x4484ac80] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x434d5740] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. Set AVStream.time_base instead.
[wav @ 0x434d5740] Using AVStream.codec.time_base as a timebase hint to the muxer is deprecated. 

# Convert to DAC

In [67]:

force_mkdir("testdata/dac-train")
force_mkdir("testdata/dac-val")
a = os.system("python3 -m dac encode wavs --model_bitrate 8kbps --n_quantizers 4 --output testdata/dac-train")
a = os.system("python3 -m dac encode wavs --model_bitrate 8kbps --n_quantizers 4 --output testdata/dac-val")
print(a)
print("Total length: ", total_length, " seconds of audio (", total_length / 3600, " hours)")


2025-03-07 17:56:05.942698: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1741366565.955625  214631 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1741366565.959595  214631 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-07 17:56:05.974453: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/angel/anaconda3/envs/valerio/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:143: FutureWarning: `torch.nn.u

0
Total length:  2597.2519047619035  seconds of audio ( 0.721458862433862  hours)


# Generate train and validation excels

In [69]:
entries = list()
for filename, class_name in mapping2.items():
    filename = os.path.basename(filename)
    filename = filename.replace(" ", "_")
    filename = filename[:-4]
    entries.append({"Full File Name": f"{filename}" + ".dac", 
                    "Class Name": f"{class_name}",
                    "Param1": 1,
                    "Param2": 2,
                    "Param3": 3})
    

pd.DataFrame(entries).to_excel("testdata/dac-train.xlsx")
pd.DataFrame(entries).to_excel("testdata/dac-val.xlsx")

In [70]:
"wavs/Helicopter_Flyover_441000.wav"[:-4] + ".dac"

'wavs/Helicopter_Flyover_441000.dac'